In [2]:
!pip install psycopg2

!pip install wikipedia

!pip install spacy

    100% |████████████████████████████████| 2.6MB 259kB/s eta 0:00:01


In [7]:
import request_category as rc
import database as db
from download import download
import search
import classify

#import pickle
from pathlib import Path
import re

import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelBinarizer, LabelEncoder

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
import sklearn.externals.joblib

In [ ]:
download( ['machine learning','business software'], depth = 1) #pckle = False

No pickle yet
Gathering page information from Category: machine learning, pages from nested sub-categories (+1 levels) will be 
included as a union for each category.
	Total categories after recursive search: 45
	Requesting 1064 unique articles - ETA: 11.73 minutes
Category DataFrames are pickled at the following location: ./pickles/machine_learning_dfs1.p. Raw text is available in
 category_pages_df in the pickle file
		Page collection  took a total of 13.67 minutes
			Updating pages table
			Updating categories table
			Updating subcategories table
			Updating page-category (link) table
___________________________________________________________________________
No pickle yet
Gathering page information from Category: business software, pages from nested sub-categories (+1 levels) will be 
included as a union for each category.
	Total categories after recursive search: 79
	Requesting 2752 unique articles - ETA: 30.33 minutes


In [16]:
db.query_to_dataframe("SELECT COUNT(*) FROM pages;")  #  LIMIT 1

,count
0,3816


In [17]:
db.query_to_dataframe("SELECT COUNT(*) FROM categories;") # LIMIT 1

,count
0,123


In [18]:
db.query_to_dataframe("SELECT COUNT(*) FROM subcategories;") #  LIMIT 1

,count
0,321


In [19]:
db.query_to_dataframe("SELECT COUNT(*) FROM page_category;") # LIMIT 1

,count
0,11100


In [20]:
query_all = """SELECT crap.category, crap.subcategory, crap.pageid, p.title, p.article
            FROM (SELECT stuff.category, stuff.subcategory, pc.pageid 
                    FROM ( SELECT category, subcategory, subcategory_id 
                            FROM subcategories sc 
                            JOIN categories c 
                            ON sc.category_id = c.category_id ) as stuff 
                    JOIN page_category pc ON stuff.subcategory_id = pc.subcategory_id) as crap 
            INNER JOIN pages p 
            ON crap.pageid = p.pageid;"""

query_all = re.sub('\s+', ' ', query_all)
all_df = db.query_to_dataframe( query_all)
all_df.shape

(11121, 5)

In [ ]:
inner_query = """SELECT scc.category, scc.subcategory, scc.subcategory_id, pc.pageid  
    FROM page_category pc
    LEFT JOIN ( SELECT category, sc.subcategory, sc.subcategory_id 
                FROM subcategories sc
                LEFT JOIN categories c
                ON sc.category_id = c.category_id
                WHERE category = 'machine learning') scc
    ON pc.subcategory_id = scc.subcategory_id"""

query = """SELECT i.category, i.subcategory, COUNT( i.pageid) 
    FROM ({}) as i
    INNER JOIN pages p
    ON i.pageid = p.pageid
    GROUP BY i.category, i.subcategory""".format(inner_query)  #COUNT(*)

query = re.sub('\s+', ' ', query)

ml_df = db.query_to_dataframe( query)
ml_df.head(50)  ## Currently database includes articles at level 0 (machine learning),  pages from subcategories